In [18]:
import os
import json
import yaml
import copy
import cv2
import numpy as np

IMAGE_FORMAT = ['jpg', 'jpeg', 'png']

In [5]:
dataroot_dir = 'd:/DATASET/MS-COCO/coco128/'

In [27]:
phase = 'train'
yolo_formatter = {}

yolo_formatter['licenses'] = {
    'name': '',
    'id': 0,
    'url': ''
}
yolo_formatter['info'] = {
    'contributor': '',
    'date_created': '',
    'description': '',
    'url': '',
    'version': '',
    'year': ''
}

In [28]:
# images
yolo_formatter['images'] = []
img_dir = os.path.join(dataroot_dir, 'images', phase)

img_id = 0
image2info = {}
for img_file in os.listdir(img_dir):
    x = {}
    if img_file.split('.')[-1].lower() in IMAGE_FORMAT:
        file_name = img_file
        img = cv2.imread(os.path.join(img_dir, img_file))
        height = img.shape[0]
        width = img.shape[1]
        x['file_name'] = file_name
        x['height'] = height
        x['width'] = width
        x['id'] = img_id
        
        x['license'] = 0
        x['flickr_url'] = ''
        x['coco_url'] = ''
        x['date_captured'] = 0
        
        yolo_formatter['images'].append(x)
        image2info[file_name] = (img_id, height, width)
        
        img_id += 1

In [29]:
# annotations
yolo_formatter['annotations'] = []

def xcycwhn_to_x1y1wh(xywhn, img_w, img_h):
    bbox = copy.deepcopy(xywhn)
    bbox[0::2] *= img_w
    bbox[1::2] *= img_h
    bbox[0] -= bbox[2]/2
    bbox[1] -= bbox[3]/2
    return list(bbox.round(2))

lbl_dir = os.path.join(dataroot_dir, 'labels', phase)
lbl_id = 0
for lbl_file in os.listdir(lbl_dir):
    if lbl_file.split(".")[-1] == 'txt':
        img_file = lbl_file.split('.')[0] + '.jpg'
        with open(os.path.join(lbl_dir, lbl_file), mode='r') as f:
            items = f.read().splitlines()
        for item in items:
            item_list = item.split(' ')
            img_id = image2info[img_file][0]
            img_h, img_w = image2info[img_file][1:]

            x = {}
            x['id'] = lbl_id
            x['image_id'] = img_id
            x['bbox'] = xcycwhn_to_x1y1wh(np.array(item_list[1:5], dtype=float), img_w, img_h)
            x['area'] = x['bbox'][2] * x['bbox'][3] 
            x['iscrowd'] = 0
            x['category_id'] = int(item_list[0])
            x['segmentation'] = []
            x['attributes'] = {
                'type': ' ',
                'occluded': False
            }
            
            yolo_formatter['annotations'].append(x)
            lbl_id += 1

In [30]:
data_path = './data/coco128.yml'
with open(data_path) as f:
    data_item = yaml.load(f, Loader=yaml.FullLoader)
    
classname_list = data_item['NAMES']

In [31]:
# categories
yolo_formatter['categories'] = []

for idx, category_name in enumerate(classname_list):
    x = {}
    x['id'] = idx
    x['supercategory'] = ''
    x['name'] = category_name
    yolo_formatter['categories'].append(x)

In [32]:
yolo_formatter['categories']

[{'id': 0, 'supercategory': '', 'name': 'person'},
 {'id': 1, 'supercategory': '', 'name': 'bicycle'},
 {'id': 2, 'supercategory': '', 'name': 'car'},
 {'id': 3, 'supercategory': '', 'name': 'motorcycle'},
 {'id': 4, 'supercategory': '', 'name': 'airplane'},
 {'id': 5, 'supercategory': '', 'name': 'bus'},
 {'id': 6, 'supercategory': '', 'name': 'train'},
 {'id': 7, 'supercategory': '', 'name': 'truck'},
 {'id': 8, 'supercategory': '', 'name': 'boat'},
 {'id': 9, 'supercategory': '', 'name': 'traffic light'},
 {'id': 10, 'supercategory': '', 'name': 'fire hydrant'},
 {'id': 11, 'supercategory': '', 'name': 'stop sign'},
 {'id': 12, 'supercategory': '', 'name': 'parking meter'},
 {'id': 13, 'supercategory': '', 'name': 'bench'},
 {'id': 14, 'supercategory': '', 'name': 'bird'},
 {'id': 15, 'supercategory': '', 'name': 'cat'},
 {'id': 16, 'supercategory': '', 'name': 'dog'},
 {'id': 17, 'supercategory': '', 'name': 'horse'},
 {'id': 18, 'supercategory': '', 'name': 'sheep'},
 {'id': 19, '

In [33]:
save_path = f'./coco128_eval_{phase}.json'
with open(save_path, 'w') as outfile:
    json.dump(yolo_formatter, outfile)